# 네이버 뉴스 크롤링(사회)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from urllib.request import urlretrieve
import re
posts = []
for i in range(1, 11):
    url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102#&date=%2000:00:00&page={i}'
    driver = webdriver.Chrome()
    driver.get(url)
    post_list = driver.find_elements(By.XPATH, '//*[@id="section_body"]/ul/li')
    for i in post_list:
        post = i.find_element(By.XPATH, 'dl/dt[2]/a')
        news_title = post.text
        link = post.get_attribute('href')
        try:
            news_image = i.find_element(By.XPATH, 'dl/dt[1]/a/img')
            news_image_url = news_image.get_attribute('src')
            # 정규표현식으로 숫자만 추출하여 파일 이름 생성
            match = re.search(r'(\d+)\.jpg', news_image_url)
            if match:
                news_image_name = match.group(1) + '.jpg'
            else:
                news_image_name = '' # 추출 실패 시 빈 문자열로 설정
            print(news_image_name)
            news_image = news_image_url.replace('https', 'http')
            news_image_path = f'C:/Users/user/Desktop/news_crawling/news_image/{news_image_name}'
            urlretrieve(news_image, news_image_path)
        except:
            news_image_path = ''
        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[1]) # 인터넷 탭 1번째로 전환
        time.sleep(1)
        create_dt = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div[1]/span').get_attribute('data-date-time')
        print(create_dt)
        news_name = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[1]').get_attribute('title')
        print(news_name)
        post_origin = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
        post_origin_text = post_origin.text
        posts.append([create_dt,news_name,'society', news_image_path, news_title, post_origin_text, link])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    driver.quit()
        
        
        
        
        

5636628.jpg
2023-12-12 20:08:01
이데일리
4924650.jpg
2023-12-12 20:05:01
한국경제
4924649.jpg
2023-12-12 20:02:01
한국경제
774952.jpg
2023-12-12 20:01:01
한국일보
3416054.jpg
2023-12-12 20:01:01
서울신문
1317969.jpg
2023-12-12 19:59:34
MBC
4924647.jpg
2023-12-12 19:56:01
한국경제
3327994.jpg
2023-12-12 19:54:08
중앙일보
263579.jpg
2023-12-12 19:49:35
채널A
4273475.jpg
2023-12-12 19:48:01
서울경제
851330.jpg
2023-12-12 19:46:01
매일신문
2415644.jpg
2023-12-12 19:40:01
오마이뉴스
1246313.jpg
2023-12-12 19:39:01
부산일보
1786538.jpg
2023-12-12 19:37:01
MBN
634051.jpg
2023-12-12 19:36:03
연합뉴스TV
254386.jpg
2023-12-12 19:36:03
더팩트
1786537.jpg
2023-12-12 19:35:01
MBN
263572.jpg
2023-12-12 19:33:48
채널A
7230906.jpg
2023-12-12 19:33:43
뉴스1
263571.jpg
2023-12-12 19:33:35
채널A
12262876.jpg
2023-12-12 19:32:06
뉴시스
263570.jpg
2023-12-12 19:31:35
채널A
1786535.jpg
2023-12-12 19:31:01
MBN
1786534.jpg
2023-12-12 19:29:01
MBN
263569.jpg
2023-12-12 19:25:55
채널A
1786532.jpg
2023-12-12 19:25:01
MBN
5229126.jpg
2023-12-12 19:24:01
매일경제
11620720.jpg
2023-12

# 데이터 베이스 연결

In [20]:
import psycopg2
from datetime import datetime
# 여러분의 데이터베이스 연결 정보
db_connection = psycopg2.connect(
    host="localhost",
    dbname='final_project',
    user='postgres',
    password="******",
    port=5432
)
# 데이터베이스 커서 생성
cursor = db_connection.cursor()
for news_item in posts:
    try:
        cursor.execute("""
            INSERT INTO final_project.society
            (create_dt, news_name, news_genre, news_image_path, news_title, news_origin, news_url, news_summary)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (news_url) DO NOTHING;
        """, (news_item[0], news_item[1], news_item[2], news_item[3], news_item[4], news_item[5], news_item[6], ''))
    except Exception as e:
        print(f"Error inserting news_item: {news_item}")
        print(f"Error message: {e}")
# 변경사항을 저장
db_connection.commit()
# 연결 종료
cursor.close()
db_connection.close()

In [21]:
db_connection = psycopg2.connect(
    host="localhost",
    dbname="final_project",
    user="postgres",
    password="******",
    port=5432
)

cursor = db_connection.cursor()
# SELECT 쿼리 실행
cursor.execute("SELECT * FROM final_project.society;")
# 결과 가져오기
result = cursor.fetchall()
# 결과 출력 (예시로 처음 5개만 출력)
# 연결 종료
cursor.close()
db_connection.close()

# 데이터 프레임으로 불러오기

In [22]:
import pandas as pd

In [23]:
pd.DataFrame(result, columns=['id', 'create_dt', 'news_name', 'news_genre', 'news_image_path', 'news_title', 'news_origin', 'news_url', 'news_summary'])

,id,create_dt,news_name,news_genre,news_image_path,news_title,news_origin,news_url,news_summary
0,1,2023-12-12 20:08:01,이데일리,society,C:/Users/user/Desktop/news_crawling/news_image...,한달 새 미성년자 4명 성폭행...휴대폰도 빼앗은 인면수심,[이데일리 김혜선 기자] 한 달 새 미성년자를 4명 성폭행하고 피해자의 휴대폰을 빼...,https://n.news.naver.com/mnews/article/018/000...,
1,2,2023-12-12 20:05:01,한국경제,society,C:/Users/user/Desktop/news_crawling/news_image...,"01년생 대학생·교수 불륜 카톡 논란…""궁디팡팡 해줘""",상대는 82년생 남자 교수로 알려져\n사진=게티이미지뱅크\n\n최근 온라인 커뮤니티...,https://n.news.naver.com/mnews/article/015/000...,
2,3,2023-12-12 20:02:01,한국경제,society,C:/Users/user/Desktop/news_crawling/news_image...,"""음식 뱉으니 손톱이"" 경악…맛집도 못 피한 '이물질 논란' [이슈+]",배달 음식 위생 논란 지적부터\n이물질 악용한 '환불 거지'까지\n업주·소비자 각 ...,https://n.news.naver.com/mnews/article/015/000...,
3,4,2023-12-12 20:01:01,한국일보,society,C:/Users/user/Desktop/news_crawling/news_image...,"대법 ""새벽 근무 거부 워킹맘 해고 부당""... '일·가정 양립 배려 의무' 첫 인정","1·6세 워킹맘 새벽 근무 거부\n사측, 2017년 채용 거부해\n6년 6개월 만 ...",https://n.news.naver.com/mnews/article/469/000...,
4,5,2023-12-12 20:01:01,서울신문,society,C:/Users/user/Desktop/news_crawling/news_image...,"경찰, ‘채 상병 사망 사건’ 임성근 전 해병대 1사단장 입건",임성근 전 해병대 1사단장이 피의자로 인건됐다. 사진은 해병대원들이 수색 중 급류에...,https://n.news.naver.com/mnews/article/081/000...,
...,...,...,...,...,...,...,...,...,...
190,196,2023-12-12 17:53:40,머니투데이,society,C:/Users/user/Desktop/news_crawling/news_image...,국내 첫 가상자산 특허소송 승리로 이끈 김앤장,"[제6회 대한민국 법무대상-송무대상]김앤장 법률사무소 김종석 변호사, 이금욱·양동현...",https://n.news.naver.com/mnews/article/008/000...,
191,197,2023-12-12 17:53:08,매일경제,society,C:/Users/user/Desktop/news_crawling/news_image...,"김동연 “경기북도, 정부가 어떤 결정 하든 담대하게 추진”","경기도, 국회서 ‘경기북부특별자치도 국회토론’ 개최\n국회의원 45명 공동주최…국회...",https://n.news.naver.com/mnews/article/009/000...,
192,198,2023-12-12 17:53:01,데일리안,society,C:/Users/user/Desktop/news_crawling/news_image...,대구 중구 클럽서 '마약파티' 벌인 20대 직원·손님 12명 검거,대구 중구 소재 클럽서 케타민 유통 및 투약한 혐의…클럽 주변서 거래 후 투약해\n...,https://n.news.naver.com/mnews/article/119/000...,
193,199,2023-12-12 17:52:52,아시아경제,society,C:/Users/user/Desktop/news_crawling/news_image...,기초연금 탐내 의붓어머니 살해 후 암매장한 40대 구속기소,70대 의붓어머니의 기초연금 자산을 탐내다 살해 후 시신을 암매장한 혐의를 받는 4...,https://n.news.naver.com/mnews/article/277/000...,
